# Install Packages

## Install Package Options

In [19]:
plot_mode='inline'    # Options: 'always' (always show plots), 'inline' (only in Jupyter/Interactive Window), 'never' (silent)
device_opt='cuda' # Options: 'sense', cuda', 'cpu'. Set to 'sense' to set to 'cpu' if available, else 'cpu'.

# Colab / Local Setup Options ##
skip_local_package_installs=True # Local only: skip package checks and pip installs, just reload code. Set to True after first run for faster startup.
ray_tune_version=None # Optional: Pin Ray Tune to specific version (e.g., '2.9.0'). Set to None to use latest.
skip_colab_git_update=False # Colab only: Skip git pull when setting up repository. Useful if you have uncommitted changes or git operations fail.
setup_mode='walk' # Local only: 'walk' for module walking or 'install' for editable install from local repo.
base_repo_path=None # Optional: override repo root detection (useful for notebooks).

## Github Repository for Functions & Classes ##
github_username='petercl8'
repo_name='FlexCNN_for_Medical_Physics'

## Directories ##
project_colab_dirPath = '/content/drive/MyDrive/Colab/Working/'     # Directory, relative to which all other directories are specified (if working on Colab)
project_local_dirPath = r'C:\Users\Peter Lindstrom\My Drive (lindstrom.peter@gmail.com)\Colab\Working'  # Directory, relative to which all other directories are specified (if working Locally)

local_repo_dirPath =  r'C:\FlexCNN_cloned'

data_dirName = 'dataset-sets'      # Dataset directory, placed in project directory (above)
plot_dirName=  'plots'             # Plots Directory, placed in project directory (above)
checkpoint_dirName='checkpoints'   # If not using Ray Tune (not tuning), PyTorch saves and loads checkpoint file from here
                                   # All checkpoint files (for training, testing, visualizing) save the states for a particular network.
                                   # Therefore, the hyperparameters for the loaded CNN must match the data in the checkpoint file.
num_examples=-1                    # Number of examples from dataset to load. Set to -1 to use all examples (this is the default)


## Perform Installs

In [20]:
import os, sys, glob, importlib, inspect, types, subprocess, pkgutil

def sense_colab():
    try:
        import google.colab
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False
    return IN_COLAB

def sense_device(device='sense'):
    if device == 'sense':
        if torch.cuda.is_available():
            device = 'cuda'
        else:
            device = 'cpu'
    elif device == 'cpu':
        device = 'cpu'
    elif device == 'cuda':
        device = 'cuda'
    return device

def install_packages(IN_COLAB=True, force_reinstall=False, include_optional=True, ray_version=None):
    """
    Installs required Python packages efficiently.
    - Detects if running in Colab or locally.
    - Installs missing packages only (unless force_reinstall=True).
    - For local: always installs CUDA-enabled PyTorch (cu124).
    - Optionally pin Ray version with ray_version (e.g., "2.9.0").
    """

    # Base list of non-PyTorch packages
    other_packages = [
        "ray[tune]", "tensorboardX", "hyperopt", "optuna",
        "numpy", "pandas", "matplotlib",
        "scikit-image", "scipy"
    ]

    # Optional packages
    optional_packages = ["tensorboard"]
    widgets_packages = ["ipywidgets"]

    missing = []

    # On Colab, just use standard installation
    if IN_COLAB:
        packages = [
            "torch", "torchvision", "torchaudio",
            "ray[tune]", "tensorboardX", "hyperopt", "optuna",
            "numpy", "pandas", "matplotlib",
            "scikit-image", "scipy"
        ]
        optional_packages_to_install = ["tensorboard"] if include_optional else []
        widgets_packages_to_install = ["ipywidgets"] if include_optional else []
        
        for pkg in packages + optional_packages_to_install + widgets_packages_to_install:
            pkg_name = pkg.split("[")[0]
            if pkg_name == "ray":
                try:
                    import ray
                    import ray.tune
                    ray_tune_installed = True
                except ImportError:
                    ray_tune_installed = False
                
                # Pin Ray version if specified
                if ray_version:
                    pkg = f"ray[tune]=={ray_version}"
                
                if force_reinstall or not ray_tune_installed:
                    missing.append(pkg)
            elif importlib.util.find_spec(pkg_name) is None or force_reinstall:
                missing.append(pkg)
        
        if not missing:
            print("✅ All required packages already installed.")
            return
        
        print(f"📦 Installing missing packages: {', '.join(missing)}")
        
        # For Colab, install PyTorch with CUDA support (cu124 works on Colab)
        torch_packages = [p for p in missing if p.split("[")[0] in ["torch", "torchvision", "torchaudio"]]
        other_missing = [p for p in missing if p.split("[")[0] not in ["torch", "torchvision", "torchaudio"]]
        
        if torch_packages:
            print(f"📦 Installing PyTorch with CUDA (cu124) for Colab GPU support...")
            cmd_torch = [sys.executable, "-m", "pip", "install", "--upgrade", "--index-url", "https://download.pytorch.org/whl/cu124"] + torch_packages
            try:
                subprocess.check_call(cmd_torch)
                print("✅ PyTorch installation complete.")
            except subprocess.CalledProcessError as e:
                print(f"❌ PyTorch installation failed: {e}")
                return
        
        if other_missing:
            print(f"📦 Installing other packages...")
            try:
                cmd = [sys.executable, "-m", "pip", "install", "--upgrade"] + other_missing
                subprocess.check_call(cmd)
                print("✅ Installation complete.")
            except subprocess.CalledProcessError as e:
                print(f"❌ Installation failed: {e}")
                print("🔁 Retrying installs individually (no cache)...")
                failed = []
                for pkg in other_missing:
                    try:
                        subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "--no-cache-dir", pkg])
                    except subprocess.CalledProcessError:
                        failed.append(pkg)
                if failed:
                    raise RuntimeError(f"Failed to install packages: {failed}")

        # Validate Ray is available before importing package modules
        try:
            import ray  # noqa: F401
            import ray.tune  # noqa: F401
        except ImportError as e:
            raise RuntimeError("Ray Tune is required but not installed. Please re-run the install cell.") from e
        return

    # Local: Always install CUDA PyTorch (cu124), other packages with standard PyPI
    print("🖥️  Local environment detected. Installing CUDA-enabled PyTorch...")
    
    torch_packages = ["torch", "torchvision", "torchaudio"]
    
    # Check which packages are missing
    for pkg in other_packages:
        pkg_name = pkg.split("[")[0]
        if pkg_name == "ray":
            try:
                import ray
                import ray.tune
                ray_tune_installed = True
            except ImportError:
                ray_tune_installed = False
            
            # Pin Ray version if specified
            if ray_version:
                pkg = f"ray[tune]=={ray_version}"
            
            if force_reinstall or not ray_tune_installed:
                missing.append(pkg)
        elif importlib.util.find_spec(pkg_name) is None or force_reinstall:
            missing.append(pkg)
    
    if include_optional:
        missing += optional_packages + widgets_packages
    
    missing = list(dict.fromkeys(missing))
    
    # Install torch with CUDA index
    print(f"📦 Installing PyTorch with CUDA (cu124)...")
    cmd_torch = [sys.executable, "-m", "pip", "install", "--upgrade", "--index-url", "https://download.pytorch.org/whl/cu124"] + torch_packages
    try:
        subprocess.check_call(cmd_torch)
        print("✅ PyTorch installation complete.")
    except subprocess.CalledProcessError as e:
        print(f"❌ PyTorch installation failed: {e}")
        return
    
    # Install other packages with standard PyPI
    if missing:
        print(f"📦 Installing other packages...")
        cmd_other = [sys.executable, "-m", "pip", "install", "--upgrade"] + missing
        try:
            subprocess.check_call(cmd_other)
            print("✅ Other packages installation complete.")
        except subprocess.CalledProcessError as e:
            print(f"❌ Other packages installation failed: {e}")
    
    # Diagnose CUDA
    print("\n" + "="*60)
    print("CUDA Diagnostic Information:")
    print("="*60)
    try:
        import torch
        print(f"✅ PyTorch version: {torch.__version__}")
        print(f"✅ CUDA available: {torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"✅ CUDA device count: {torch.cuda.device_count()}")
            for i in range(torch.cuda.device_count()):
                print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")
        else:
            print("❌ CUDA is NOT available - this is a problem!")
            print("   Checking nvidia-smi...")
            try:
                result = subprocess.check_output("nvidia-smi", shell=True).decode()
                print("   nvidia-smi output:")
                for line in result.split('\n')[:10]:
                    print(f"     {line}")
            except Exception as e:
                print(f"   nvidia-smi not found: {e}")
    except ImportError as e:
        print(f"❌ PyTorch import failed: {e}")
    print("="*60 + "\n")

# Reload_submodules is used in both setup functions
def reload_submodules(pkg):
    """Reload all submodules in a package to pick up code changes."""
    for importer, modname, ispkg in pkgutil.walk_packages(pkg.__path__, pkg.__name__ + "."):
        try:
            sub_module = importlib.import_module(modname)
            importlib.reload(sub_module)
        except Exception:
            pass

def resolve_repo_root(base_repo_path=None):
    """Resolve repo root by searching for setup.py/pyproject.toml."""
    if base_repo_path:
        return base_repo_path

    try:
        start_dir = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        start_dir = os.getcwd()

    repo_root = start_dir
    while repo_root != os.path.dirname(repo_root):
        if os.path.exists(os.path.join(repo_root, "setup.py")) or os.path.exists(os.path.join(repo_root, "pyproject.toml")):
            return repo_root
        repo_root = os.path.dirname(repo_root)

    raise FileNotFoundError("Could not locate repo root (setup.py or pyproject.toml). Set base_repo_path.")

def setup_colab_environment(
    github_username: str = "peterlabcl8",
    repo_name: str = "FlexCNN_for_Medical_Physics",
    local_repo_path: str = None,
    skip_git_update: bool = False,
    verbose: bool = True):
    """
    Setup environment for Colab: clone/pull repo and install via pip.
    Injects all package symbols into caller's globals.
    
    Args:
        github_username: GitHub username for the repository
        repo_name: Repository name
        local_repo_path: Local path (unused for Colab, kept for consistency)
        skip_git_update: If True, skip git pull (useful if already up-to-date or if git operations fail)
        verbose: Print status messages
    """
    # Determine base directory
    base_dir = "/content"
    repo_path = os.path.join(base_dir, repo_name)
    repo_url = f"https://github.com/{github_username}/{repo_name}.git"

    # Clone or update
    if not os.path.exists(repo_path):
        if verbose:
            print(f"📦 Cloning {repo_name} into {base_dir}...")
        try:
            subprocess.run(["git", "clone", repo_url], cwd=base_dir, check=True)
        except subprocess.CalledProcessError as e:
            print(f"⚠️  Git clone failed: {e}")
            print(f"   Proceeding without updating repository...")
    elif not skip_git_update:
        if verbose:
            print(f"🔄 Pulling latest changes in {repo_path}...")
        try:
            subprocess.run(["git", "pull"], cwd=repo_path, check=True)
        except subprocess.CalledProcessError as e:
            print(f"⚠️  Git pull failed: {e}")
            print(f"   Proceeding with existing repository...")
    else:
        if verbose:
            print(f"⏭️  Skipping git update (skip_git_update=True)")

    # Install package in editable mode
    if verbose:
        print("⚙️ Installing the package in editable mode...")
    try:
        subprocess.run([sys.executable, "-m", "pip", "install", "-e", "."],
                       cwd=repo_path, check=True)
    except subprocess.CalledProcessError as e:
        print(f"⚠️  Package installation failed: {e}")
        print(f"   Attempting to proceed anyway...")

    # Ensure repo path is importable
    if repo_path not in sys.path:
        sys.path.insert(0, repo_path)

    # Import the package
    package = importlib.import_module(repo_name)

    # Reload all submodules
    reload_submodules(package)

    # Gather all symbols
    imported = {}
    for _, modname, ispkg in pkgutil.walk_packages(package.__path__, package.__name__ + "."):
        mod = importlib.import_module(modname)
        for name, obj in inspect.getmembers(mod):
            if not name.startswith("_"):
                imported[name] = obj

    # Inject symbols into caller's globals
    if verbose:
        print("✨ Injecting all symbols into global namespace...")
    caller_globals = inspect.stack()[1].frame.f_globals
    caller_globals.update(imported)
    if verbose:
        print(f"✅ Setup complete: {len(imported)} symbols loaded into globals.")

def setup_local_environment(
    repo_name: str = "FlexCNN_for_Medical_Physics",
    mode: str = "walk",
    base_repo_path: str = None,
    verbose: bool = True):
    """
    Setup environment for local machine: install or walk package.
    Injects all package symbols into caller's globals.
    """
    if mode not in ("walk", "install"):
        raise ValueError(f"setup_mode must be 'walk' or 'install', got '{mode}'.")

    package_root = resolve_repo_root(base_repo_path)

    if mode == "install":
        if verbose:
            print(f"⚙️ Installing the package in editable mode from {package_root}...")
        try:
            subprocess.run([sys.executable, "-m", "pip", "install", "-e", "."],
                           cwd=package_root, check=True)
        except subprocess.CalledProcessError as e:
            print(f"⚠️  Package installation failed: {e}")
            print("   Attempting to proceed anyway...")
    
    # Add to sys.path
    if package_root not in sys.path:
        sys.path.insert(0, package_root)
        if verbose:
            print(f"📂 Added {package_root} to sys.path")

    # Import and walk the package
    if verbose:
        print(f"📦 Loading {repo_name} package ({mode} mode)...")
    package = importlib.import_module(repo_name)
    
    # Reload all submodules to pick up code changes
    reload_submodules(package)
    
    # Gather all symbols from all modules
    imported = {}
    for _, modname, ispkg in pkgutil.walk_packages(package.__path__, package.__name__ + "."):
        try:
            mod = importlib.import_module(modname)
            imported.update({name: obj for name, obj in vars(mod).items() if not name.startswith('_')})
        except Exception:
            pass
    
    # Inject symbols into caller's globals
    if verbose:
        print("✨ Injecting all symbols into global namespace...")
    caller_globals = inspect.stack()[1].frame.f_globals
    caller_globals.update(imported)
    if verbose:
        print(f"✅ Setup complete: {len(imported)} symbols loaded into globals.")

def refresh_repo(
    IN_COLAB = True,
    repo_name: str = "FlexCNN_for_Medical_Physics",
    github_username: str = "petercl8",
    local_repo_path: str = None,
    auto_import: bool = True,
    verbose: bool = True):
    """
    Clone/pull and install the repo, then optionally auto-import all modules.
    Also reloads all submodules to reflect changes without restarting the runtime.
    """
    # --- Determine base directory ---
    base_dir = "/content" if IN_COLAB else local_repo_path
    if base_dir is None:
        raise ValueError("local_repo_path must be provided if not in Colab")

    repo_path = os.path.join(base_dir, repo_name)
    repo_url = (
        f"https://github.com/{github_username}/{repo_name}.git"
        if IN_COLAB
        else f"git@github.com:{github_username}/{repo_name}.git"
    )

    # --- Clone or update ---
    if not os.path.exists(repo_path):
        if verbose:
            print(f"📦 Cloning {repo_name} into {base_dir}...")
        subprocess.run(["git", "clone", repo_url], cwd=base_dir, check=True)
    else:
        if verbose:
            print(f"🔄 Pulling latest changes in {repo_path}...")
        subprocess.run(["git", "pull"], cwd=repo_path, check=True)

    # --- Install package in editable mode ---
    if verbose:
        print("⚙️ Installing the package in editable mode...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-e", "."],
                   cwd=repo_path, check=True)

    # --- Ensure repo path is importable ---
    if repo_path not in sys.path:
        sys.path.insert(0, repo_path)

    # --- Import the package ---
    package = importlib.import_module(repo_name)

    # --- Reload all submodules recursively ---
    def reload_submodules(pkg):
        for _, modname, ispkg in pkgutil.walk_packages(pkg.__path__, pkg.__name__ + "."):
            if modname in sys.modules:
                importlib.reload(sys.modules[modname])
            else:
                importlib.import_module(modname)
        importlib.reload(pkg)

    reload_submodules(package)

    # --- Gather all symbols ---
    imported = {}
    for _, modname, ispkg in pkgutil.walk_packages(package.__path__, package.__name__ + "."):
        mod = importlib.import_module(modname)
        for name, obj in inspect.getmembers(mod):
            if not name.startswith("_"):
                imported[name] = obj

    # --- Inject symbols into caller's globals if requested ---
    if auto_import:
        if verbose:
            print("✨ Injecting all symbols into global namespace...")
        caller_globals = inspect.stack()[1].frame.f_globals
        caller_globals.update(imported)
        if verbose:
            print(f"✅ Setup complete: {len(imported)} symbols loaded into globals.")
    else:
        if verbose:
            print(f"✅ Imported {len(imported)} symbols (not injected).")

"""## Install Packages & Setup Repo"""

# --- Sense environment ---
IN_COLAB = sense_colab()

# --- Environment-aware install + repo refresh ---
if IN_COLAB:
    # Colab: install deps, then clone/pull and reload
    install_packages(IN_COLAB, ray_version=ray_tune_version)
    setup_colab_environment(
        github_username=github_username,
        repo_name=repo_name,
        skip_git_update=skip_colab_git_update
    )
else:
    # Local: optionally install deps, then reload modules
    if not skip_local_package_installs:
        # Local: install missing packages (full check)
        install_packages(IN_COLAB, ray_version=ray_tune_version)
    else:
        # Local with skip_local_package_installs=True: assume packages already installed
        print("⏭️  Skipping package checks (skip_local_package_installs=True)")
    setup_local_environment(repo_name=repo_name, mode=setup_mode, base_repo_path=base_repo_path)


# --- Test Resources ---
list_compute_resources()

# --- Set main project directory ---
project_dirPath = setup_project_dirs(IN_COLAB, project_local_dirPath, project_colab_dirPath, mount_colab_drive=True)

# --- Set Device ---
device = sense_device(device=device_opt)

# --- Configure Plotting ---
from FlexCNN_for_Medical_Physics.functions.helper.display_images import configure_plotting
configure_plotting(plot_mode=plot_mode)

# Build grouped parameter dictionaries #

common_settings = {
    'run_mode': run_mode,
    'device': device,
    'use_cache': use_cache,
    'cache_max_gb': cache_max_gb,
    'cache_dir': cache_dir,
    'num_examples': num_examples,
    'act_recon1_scale': act_recon1_scale,
    'act_recon2_scale': act_recon2_scale,
    'act_sino_scale': act_sino_scale,
    'act_image_scale': act_image_scale,
    'atten_image_scale': atten_image_scale,
    'atten_sino_scale': atten_sino_scale,
}

base_dirs = {
    'project_dirPath': project_dirPath,
    'plot_dirName': plot_dirName,
    'checkpoint_dirName': checkpoint_dirName,
    'tune_storage_dirName': tune_storage_dirName,
    'tune_dataframe_dirName': tune_dataframe_dirName,
    'test_dataframe_dirName': test_dataframe_dirName,
    'data_dirName': data_dirName
}

data_files = {
    'tune_act_sino_file': tune_act_sino_file,
    'tune_act_image_file': tune_act_image_file,
    'tune_act_recon1_file': tune_act_recon1_file,
    'tune_act_recon2_file': tune_act_recon2_file,
    'tune_atten_image_file': tune_atten_image_file,
    'tune_atten_sino_file': tune_atten_sino_file,
    'tune_val_act_sino_file': tune_val_act_sino_file,
    'tune_val_act_image_file': tune_val_act_image_file,
    'tune_val_atten_image_file': tune_val_atten_image_file,
    'tune_val_atten_sino_file': tune_val_atten_sino_file,
    'tune_qa_act_sino_file': tune_qa_act_sino_file,
    'tune_qa_act_image_file': tune_qa_act_image_file,
    'tune_qa_hotMask_file': tune_qa_hotMask_file,
    'tune_qa_hotBackgroundMask_file': tune_qa_hotBackgroundMask_file,
    'tune_qa_coldMask_file': tune_qa_coldMask_file,
    'tune_qa_coldBackgroundMask_file': tune_qa_coldBackgroundMask_file,
    'tune_qa_backMask_file': None, # Added this line to fix the KeyError
    'tune_qa_atten_image_file': tune_qa_atten_image_file,
    'tune_qa_atten_sino_file': tune_qa_atten_sino_file,
    'train_act_sino_file': train_act_sino_file,
    'train_act_image_file': train_act_image_file,
    'train_act_recon1_file': train_act_recon1_file,
    'train_act_recon2_file': train_act_recon2_file,
    'train_atten_image_file': train_atten_image_file,
    'train_atten_sino_file': train_atten_sino_file,
    'test_act_sino_file': test_act_sino_file,
    'test_act_image_file': test_act_image_file,
    'test_act_recon1_file': test_act_recon1_file,
    'test_act_recon2_file': test_act_recon2_file,
    'test_atten_image_file': test_atten_image_file,
    'test_atten_sino_file': test_atten_sino_file,
    'visualize_act_sino_file': visualize_act_sino_file,
    'visualize_act_image_file': visualize_act_image_file,
    'visualize_act_recon1_file': visualize_act_recon1_file,
    'visualize_act_recon2_file': visualize_act_recon2_file,
    'visualize_atten_image_file': visualize_atten_image_file,
    'visualize_atten_sino_file': visualize_atten_sino_file,
    }

mode_files = {
    'tune_csv_file': tune_csv_file,
    'train_checkpoint_file': train_checkpoint_file,
    'test_checkpoint_file': test_checkpoint_file,
    'test_csv_file': test_csv_file,
    'visualize_checkpoint_file': visualize_checkpoint_file
}

network_opts = {
    'network_type': network_type,
    'train_SI': train_SI,
    'gen_image_size': gen_image_size,
    'gen_sino_size': gen_sino_size,
    'gen_image_channels': gen_image_channels,
    'gen_sino_channels': gen_sino_channels,
    'SI_normalize': SI_normalize,
    'IS_normalize': IS_normalize,
}

tune_opts = {
    'tune_exp_name': tune_exp_name,
    'tune_scheduler': tune_scheduler,
    'tune_dataframe_fraction': tune_dataframe_fraction,
    'tune_restore': tune_restore,
    'tune_max_t': tune_max_t,
    'tune_minutes': tune_minutes,
    'tune_metric': tune_metric,
    'tune_even_reporting': tune_even_reporting,
    'tune_batches_per_report': tune_batches_per_report,
    'tune_examples_per_report': tune_examples_per_report,
    'tune_augment': tune_augment,
    'tune_grace_period': tune_grace_period,
    'tune_debug': tune_debug,
    'tune_force_fixed_config': tune_force_fixed_config,
    'tune_report_for': tune_report_for,
    'tune_qa_hot_weight': tune_qa_hot_weight,
    'tune_eval_batch_size': tune_eval_batch_size,
    'num_CPUs': num_CPUs,
    'num_GPUs': num_GPUs,
    'cpus_per_trial': CPUs_per_trial,  # per trial
    'gpus_per_trial': GPUs_per_trial,  # per trial
    'tune_search_alg': tune_search_alg,
}

train_opts = {
    'train_load_state': train_load_state,
    'train_save_state': train_save_state,
    'training_epochs': train_epochs,
    'train_augment': train_augment,
    'train_shuffle': train_shuffle,
    'train_display_step': train_display_step,
    'train_sample_division': train_sample_division,
    'train_show_times': train_show_times,
}

test_opts = {
    'test_display_step': test_display_step,
    'test_batch_size': test_batch_size,
    'test_chunk_size': test_chunk_size,
    'testset_size': testset_size,
    'test_begin_at': test_begin_at,
    'test_compute_MLEM': test_compute_MLEM,
    'test_merge_dataframes': test_merge_dataframes,
    'test_show_times': test_show_times,
    'test_shuffle': test_sample_division,
    'test_sample_division': test_sample_division
}

viz_opts = {
    'visualize_batch_size': visualize_batch_size,
    'visualize_offset': visualize_offset,
    'visualize_shuffle': visualize_shuffle,
}

# Build paths and settings using new functions
paths = setup_paths(
    run_mode=run_mode,
    base_dirs=base_dirs,
    data_files=data_files,
    mode_files=mode_files,
    test_ops=test_opts,
    viz_ops=viz_opts
)

settings = setup_settings(
    run_mode=run_mode,
    common_settings=common_settings,
    tune_opts=tune_opts,
    train_opts=train_opts,
    test_opts=test_opts,
    viz_opts=viz_opts,
)

# --- Build Config Dictionary ---
config = construct_config(
    run_mode=run_mode,
    network_opts=network_opts,
    tune_opts=tune_opts,
    test_opts=test_opts,
    viz_opts=viz_opts,
    config_ACT_SI=config_ACT_SI,
    config_ACT_IS=config_ACT_IS,
    config_ATTEN_SI=config_ATTEN_SI,
    config_ATTEN_IS=config_ATTEN_IS,
    config_CONCAT=config_CONCAT,
    config_FROZEN_COFLOW=config_FROZEN_COFLOW,
    config_FROZEN_COUNTERFLOW=config_FROZEN_COUNTERFLOW,
    config_GAN_SI=None,
    config_GAN_IS=None,
    config_RAY_SI=config_RAY_SI,
    config_RAY_SI_learnScale=config_RAY_SI_learnScale,
    config_RAY_SI_fixedScale=config_RAY_SI_fixedScale,
    config_RAY_IS=config_RAY_IS,
    config_RAY_IS_learnScale=config_RAY_IS_learnScale,
    config_RAY_IS_fixedScale=config_RAY_IS_fixedScale,
    config_RAY_SUP=config_RAY_SUP,
    config_RAY_SUP_FROZEN=config_RAY_SUP_FROZEN,
    config_RAY_GAN=None,
    config_RAY_GAN_CYCLE=None,
)

⏭️  Skipping package checks (skip_local_package_installs=True)
📦 Loading FlexCNN_for_Medical_Physics package (walk mode)...
✨ Injecting all symbols into global namespace...
✅ Setup complete: 244 symbols loaded into globals.
CPUs available: 20
GPUs available: 1
  GPU 0: NVIDIA GeForce RTX 3080 Ti Laptop GPU
[INFO] Plot mode: inline - using interactive backend: module://matplotlib_inline.backend_inline


# Analysis

## Compute Recon Scales



In [21]:
#avg_activity = compute_average_activity_per_image(paths, dataset='train')
#scales1 = compute_quantitative_reconstruction_scale(paths, dataset='train', compute_sinogram_scale=False)
#scales2 = analyze_reconstruction_scale_distribution(paths, dataset='train', sample_mode='full', sample_size=1000, ratio_cap_multiple=None)
#sinogram_scale = compute_sinogram_to_image_scale(paths, dataset='train', sample_number=1000)

NameError: name 'paths' is not defined

**Absolute Activities (Training Set)**

Annihilation: Average counts per image: 8676873

Activity

*   Average activity per image: 144616.84
*   Std dev activity per image: 89057.51
*   Min activity per image: 13.16
*   Max activity per image: 640712.82

Recon1: Average activity per image: 43167.45

Recon2: Average activity per image: 72402.20

Attenuation: 469.025

**Activity Sinogram Scale**

*   sample_sinogram_number=100, clip_percentile_low=10, clip_percentile_high=90, 'median': 0.341


**Quantitative Reconstruction Scales:**

Training Set:

*   FORE: 3.350
*   Oblique: 1.997
*   Atten: 308.335

Test Set:

*   FORE: 3.412
*   Oblique: 2.014

**Analysis Scales for No Cap:**

Training set:

*   FORE: 3.400 +/- 0.305 (max_ratio = 4.768)
*   Oblique: 2.025 +/- 0.077 (max_ratio = 2.658)

Test set:

*   FORE: 3.426 +/- 0.292 (max_ratio = 4.727)
*   Oblique: 2.034 +/- 0.073 (max_ratio = 2.683)

## Adjust Atten Sino Alignment

In [ ]:
# --- Refresh Repository ---
refresh_repo(IN_COLAB, local_repo_path=local_repo_dirPath)

visualize_sinogram_alignment(
    paths,
    settings,
    num_examples=3,
    scale_num_examples=3, # Set to a large number to get an accurate scale
    start_index=500,
    randomize=True,
    random_seed=None,
    fig_size=3,
    cmap='inferno',
    circle=False,
    theta_type='symmetrical', # Set to 'speed' to match activity sinogram angular sampling after pooling
                        # Set to 'symmetrical' to match sampling before pooling.
    # Activity resize/pad options
    act_resize_type='crop_pad',   # 'crop_pad', 'bilinear', or None
    act_pad_type='zeros', # 'sinoram' or 'zeros'
    act_vert_size=320,
    act_target_width=320,
    act_pool_size=1,
    # Attenuation resize/pad options
    atten_resize_type='crop_pad', # 'crop_pad', 'bilinear', or None
    atten_pad_type='zeros',
    atten_vert_size=320,
    atten_target_width=320,
    atten_pool_size=1,
)


In [ ]:
print(paths['train_sino_path'])

## Plot Example Images

In [ ]:
# --- Refresh Repository ---
#refresh_repo(IN_COLAB, local_repo_path=local_repo_dirPath)

## Anthropomorphic phantoms ##
##############################
fig_size_XCAT=2.5
indexes_XCAT = [100, 200, 300, 400, 500, 600, 700]

test_array_names = ['test-actMap.npy', 'test-obliqueImage.npy', 'test-highCountImage.npy',]
test_sino = 'test-highCountSino-180x180.npy'
#test_sino = 'test-highCountImage.npy'

train_array_names = ['train-actMap.npy','train-obliqueImage.npy', 'train-highCountImage.npy', ]
train_sino = 'train-highCountSino-180x180.npy'
#train_sino = 'train-highCountImage.npy'

## QA phantoms ##
#################
fig_size_QA=2.5
indexes_QA = [13, 14, 15, 16, 17, 18, 19]

NEMA_array_names = ['QA-NEMA-actMap.npy','QA-NEMA-obliqueImage.npy', 'QA-NEMA-highCountImage.npy',]
NEMA_sino = 'QA-NEMA-highCountSino.npy'
#NEMA_sino = 'QA-NEMA-highCountImage.npy'

Pinwheel_array_names = ['QA-Pinwheel-actMap.npy','QA-Pinwheel-obliqueImage.npy', 'QA-Pinwheel-highCountImage.npy']
Pinwheel_sino = 'QA-Pinwheel-highCountSino.npy'
#Pinwheel_sino = 'QA-Pinwheel-highCountImage.npy'

Radial_array_names = ['QA-Radial-actMap.npy','QA-Radial-obliqueImage.npy', 'QA-Radial-highCountImage.npy']
Radial_sino = 'QA-Radial-highCountSino.npy'
#Radial_sino = 'QA-Radial-highCountImage.npy'

#actMap = 'QA-Axial-actMap.npy'
#sino = 'QA-Axial-highCountSino.npy'

## Network Configs ##
#####################

checkpoint_fileName_SI = 'checkpoint-fullSet-highCountSino2actMap-tunedSSIM-augII-100epochs'
config_SI = {
   "SI_dropout": False,
  "SI_exp_kernel": 3,
  "SI_fixedScale": 1,
  "SI_gen_fill": 1,
  "SI_gen_final_activ": nn.ELU(alpha=1.0),
  "SI_gen_hidden_dim": 29,
  "SI_gen_mult": 1.5090047574838394,
  "SI_gen_neck": 11,
  "SI_gen_z_dim": 486,
  "SI_layer_norm": "instance",
  "SI_learnedScale_init": 20.45467480669682,
  "SI_normalize": False,
  "SI_pad_mode": "zeros",
  "SI_skip_mode": "none",
  "batch_base2_exponent": 6,
  "gen_b1": 0.34632557248900636,
  "gen_b2": 0.10963336318792913,
  "gen_lr": 0.0005750756280291565,
  "gen_image_channels": 1,
  "gen_image_size": 180,
  "network_type": "SUP",
  "gen_sino_channels": 3,
  "sino_size": 180,
  "sup_criterion": nn.MSELoss(),
  "train_SI": True
}

####################
### Plots Images ###
####################

#image_tensor, sino_tensor = PlotPhantomRecons(test_array_names, test_sino, config_SI, paths, indexes_XCAT, checkpoint_fileName_SI, fig_size_XCAT, device, settings)
#show_single_unmatched_tensor(sino_tensor[0:2], fig_size=10)

#image_tensor, sino_tensor = PlotPhantomRecons(NEMA_array_names, NEMA_sino, config_SI, paths, indexes_QA, checkpoint_fileName_SI, fig_size_QA, device, settings)
image_tensor, sino_tensor = PlotPhantomRecons(Pinwheel_array_names, Pinwheel_sino, config_SI, paths, indexes_QA, checkpoint_fileName_SI, fig_size_QA, device, settings)
show_single_unmatched_tensor(sino_tensor[0:2], fig_size=25)

#image_tensor, sino_tensor = PlotPhantomRecons(Radial_array_names, Radial_sino, config_SI, paths, indexes_QA, checkpoint_fileName_SI, fig_size_QA, device, settings)

#############
## Metrics ##
#############

#frame_SSIM_MLEM, placeholder = calculate_metric(MLEM_output, image_tensor, SSIM, dataframe = True, label='MLEM, SSIM')
#frame_MSE_MLEM, placeholder =  calculate_metric(MLEM_output, image_tensor, MSE, dataframe = True, label='MLEM, MSE')
#print('################### MLEM ###################')
#print(frame_SSIM_MLEM.T)
#print(frame_MSE_MLEM.T)

break


## Tuning Curves

In [ ]:
#################
## Set Options ##
#################

tune_exp_name="/content/drive/MyDrive/Colab/Working/searches/search-fullSet-highCountSino2actMap-tunedSSIM-AugmentSI"
save_fig=True

titlesize=13
fontsize=12
ticksize=10
dpi=800
figsize=(10,8)

fig = plt.figure(figsize=figsize, dpi=dpi)
gs = gridspec.GridSpec(ncols=100, nrows=100)

# Top Row Axes #
ax1 = fig.add_subplot(gs[0:25,   0:100])
ax2 = fig.add_subplot(gs[38:62,   0:100])
#ax3 = fig.add_subplot(gs[75:100,  0:100])


## Plots ###

#refresh_repo(IN_COLAB, local_repo_path=local_repo_dirPath)

result_grid, bestResult_logDir = PlotFrame(paths, tune_exp_name, ax1, 'batch_step', 'Batch Step', 'MSE', 'MSE', ylim=(50,20000), logy=True)
ax1.set_title('(A) MSE Learning Curves', fontsize=titlesize)

result_grid, bestResult_logDir = PlotFrame(paths, tune_exp_name, ax2, 'batch_step', 'Batch Step', 'SSIM', 'SSIM', ylim=(0,1), logy=False)
ax2.set_title('(B) SSIM Learning Curves', fontsize=titlesize)

#result_grid, bestResult_logDir = PlotFrame(paths, tune_exp_name, ax3, 'batch_step', 'Batch Step', 'CUSTOM', 'Local Distributions Metric', ylim=(300,500))
#ax3.set_title('(A) LDM Learning Curves', fontsize=titlesize)

# Best Result #

#print(bestResult_logdir)

# Save Fig? #

if save_fig:
    plot_save_name='figure-tuning'
    savefig(os.path.join(paths['plot_dirPath'], plot_save_name+'.svg'), bbox_inches='tight')

## Tune Frame Scatter Plots

In [ ]:
print(paths)

In [ ]:

tune_csv_file = 'frame-tunedMSE-ASHA'


tune_dataframe = pd.read_csv(os.path.join(paths['tune_dataframe_dirPath'], tune_csv_file+'.csv'))

## Describe Dataframes ##

#plt.scatter(tune_dataframe['num_params'], tune_dataframe['mean_CNN_MSE'])
#plt.scatter(tune_dataframe['num_params'][1:], tune_dataframe['mean_CNN_MSE'][1:])

tune_dataframe.plot.scatter('num_params', 'mean_CNN_MSE', ylim=(0,5))
tune_dataframe.plot.scatter('gen_lr', 'mean_CNN_MSE', ylim=(0,5))
tune_dataframe.plot.scatter('batch_size', 'mean_CNN_MSE', ylim=(0,5))

'''
plt.scatter(tune_dataframe['num_params'], tune_dataframe['mean_CNN_MSE'], ylim=(0,1))
plt.xlabel('Number of Parameters')
plt.ylabel('MSE')
plt.show()
'''

tune_dataframe.describe()

## Load: Test Dataframes


In [ ]:
# tunedMSE #
test_dataframe_dirPath1= '/content/drive/MyDrive/Colab/Working/Dataframes-TestOnFull'
#test_csv_file1 = 'combined-tunedFullMSE-trainedFull-onTrainingSet-noMLEM'   # Use this dataframe to determine thresholds for sorting training set by metrics
#test_csv_file1 = 'combined-tunedFullMSE-trainedFull-onTestSet-wMLEM'       # Use this dataframe to determine thresholds for sorting test set by metrics
#test_csv_file1 = 'combined-tunedFullSSIM-trainedFull-onTestSet-wMLEM'
test_csv_file1 = 'combined-tunedHighMSE-trainedHighMSE-onTestSet-wMLEM'

#test_dataframe_dirPath2= '/content/drive/MyDrive/Colab/Working/Dataframes-Test-Quartile-MSE'
#test_dataframe_dirPath2= '/content/drive/MyDrive/Colab/Working/Dataframes-TestOnFull'
#test_csv_file2 = 'combined-tunedFullSSIM-trainedFull-onTestSet-wMLEM'
#test_csv_file2 = 'combined-tunedHighMSE-trainedHighMSE-onTestSet-wMLEM'
test_csv_file2 = 'combined-tunedLowSSIM-trainedLowSSIM-onTestSet-wMLEM'

# Read Dataframes from File #
dataframe_path1 = os.path.join(test_dataframe_dirPath1, test_csv_file1+'.csv')
dataframe1 = pd.read_csv(dataframe_path1)
dataframe_path2 = os.path.join(test_dataframe_dirPath2, test_csv_file2+'.csv')
dataframe2 = pd.read_csv(dataframe_path2)v

## Describe Dataframes ##

#frame_picked = dataframe[dataframe["SSIM (ML-EM)"]>dataframe["SSIM (FBP)"]]
#frame_picked = dataframe[dataframe["SSIM (Network)"]>dataframe["SSIM (ML-EM)"]]

#frame_picked = dataframe[dataframe["MSE (Network)"]<dataframe["MSE (ML-EM)"]]
#frame_picked = dataframe[dataframe["MSE (ML-EM)"]<dataframe["MSE (FBP)"]]

#frame_picked = dataframe1[dataframe1["MSE (FBP)"]>0.95908]
#frame_picked = dataframe1[dataframe1["MSE (FBP)"]<0.330922]
frame_picked = dataframe1[dataframe1["SSIM (FBP)"]<0.837850]

#dataframe1.describe()
dataframe2.describe()
#frame_picked.describe()

### Plot Test Dataframes

In [ ]:
## Specify Plotting Parameters ##
plot_type = 2 # 1 = histograms, 2 = bin plots, 3 = both

column_MSE_1 = 'MSE (ML-EM)'
#column_MSE_1 = 'MSE (FBP)'
column_MSE_2 = 'MSE (Network)'
#column_MSE_2 = 'MSE (FBP)'

column_SSIM_1 = 'SSIM (ML-EM)'
#column_SSIM_1 = 'SSIM (FBP)'
column_SSIM_2 = 'SSIM (Network)'
#column_SSIM_2 = 'SSIM (FBP)'


titlesize=12
fontsize=9
ticksize=7
dpi=800

if plot_type == 1 or plot_type == 2:
    figsize=(8,6) # 17,5
    fig = plt.figure(figsize=figsize, dpi=dpi)
    gs = gridspec.GridSpec(ncols=100, nrows=100)

    # Top Row Axes #
    ax1 = fig.add_subplot(gs[0:42,   0:43])
    ax2 = fig.add_subplot(gs[0:42,   57:100])

    # Bottom Row Axes #
    ax3 = fig.add_subplot(gs[58:100, 0:43])
    ax4 = fig.add_subplot(gs[58:100, 57:100])

    if plot_type == 1:
        plot_hist_1D(ax1, dataframe1, '(1) CNN-A: MSE Histogram',  'MSE', 'frequency', column_MSE_1 , column_MSE_2, xlim=(0,4), ylim=(0,5000), bins=40)
        plot_hist_1D(ax2, dataframe1, '(2) CNN-A: SSIM Histogram', 'SSIM','frequency', column_SSIM_1, column_SSIM_2, xlim=(0.6,1), ylim=(0,4000), bins=40)
        plot_hist_1D(ax3, dataframe2, '(3) CNN-B: MSE Histogram',  'MSE', 'frequency', column_MSE_1 , column_MSE_2,  xlim=(0,4), ylim=(0,5000),  bins=40)
        plot_hist_1D(ax4, dataframe2, '(4) CNN-B: SSIM Histogram', 'SSIM','frequency', column_SSIM_1, column_SSIM_2, xlim=(0.6,1), ylim=(0,4000), bins=40)
    if plot_type == 2:
        plot_hist_2D(ax1, dataframe1, '(1) CNN-A: MSE Bin Plot', column_MSE_1, 'MSE (CNN-A)', column_MSE_1 , column_MSE_2,(0,1.5), (0,1.5), gridsize=60)
        plot_hist_2D(ax2, dataframe1, '(2) CNN-A: SSIM Bin Plot',column_SSIM_1, 'SSIM (CNN-A)', column_SSIM_1, column_SSIM_2, (.7,1), (.7,1), gridsize=100)
        plot_hist_2D(ax3, dataframe2, '(3) CNN-B: MSE Bin Plot', column_MSE_1, 'MSE (CNN-B)', column_MSE_1 , column_MSE_2, (0,1.5), (0,1.5), gridsize=60)
        plot_hist_2D(ax4, dataframe2, '(4) CNN-B: SSIM Bin Plot', column_SSIM_1, 'SSIM (CNN-B)', column_SSIM_1, column_SSIM_2, (.7,1), (.7,1), gridsize=100)

if plot_type == 3:
    figsize=(15,6) # 17,5
    fig = plt.figure(figsize=figsize, dpi=dpi)
    gs = gridspec.GridSpec(ncols=100, nrows=100)

    # Top Row Axes #
    ax1 = fig.add_subplot(gs[0:42,   0:18]) # 20
    ax2 = fig.add_subplot(gs[0:42,   25:47]) # 22
    ax3 = fig.add_subplot(gs[0:42,   53:74]) # 20
    ax4 = fig.add_subplot(gs[0:42,   80:100]) # 22

    # Bottom Row Axes #
    ax5 = fig.add_subplot(gs[58:100, 0:18]) # -5-
    ax6 = fig.add_subplot(gs[58:100, 25:47]) # -3 - -3-
    ax7 = fig.add_subplot(gs[58:100, 53:74]) # -5-
    ax8 = fig.add_subplot(gs[58:100, 80:100])

    plot_hist_1D(ax1, dataframe1, '(1) CNN-A: MSE Histogram',  'MSE', 'frequency', column_MSE_1 , column_MSE_2, xlim=(0,4), ylim=(0,5000), bins=40)
    plot_hist_1D(ax2, dataframe1, '(3) CNN-A: SSIM Histogram', 'SSIM','frequency', column_SSIM_1, column_SSIM_2, xlim=(0.6,1), ylim=(0,4000), bins=40)
    plot_hist_2D(ax3, dataframe1, '(5) CNN-A: MSE Bin Plot', column_MSE_1, 'MSE (CNN-A)', column_MSE_1 , column_MSE_2,(0,1.5), (0,1.5), gridsize=60)
    plot_hist_2D(ax4, dataframe1, '(7) CNN-A: SSIM Bin Plot',column_SSIM_1, 'SSIM (CNN-A)', column_SSIM_1, column_SSIM_2, (.7,1), (.7,1), gridsize=100)

    plot_hist_1D(ax5, dataframe2, '(2) CNN-B: MSE Histogram',  'MSE', 'frequency', column_MSE_1 , column_MSE_2,  xlim=(0,4), ylim=(0,5000),  bins=40)
    plot_hist_1D(ax6, dataframe2, '(4) CNN-B: SSIM Histogram', 'SSIM','frequency', column_SSIM_1, column_SSIM_2, xlim=(0.6,1), ylim=(0,4000), bins=40)
    plot_hist_2D(ax7, dataframe2, '(6) CNN-B: MSE Bin Plot', column_MSE_1, 'MSE (CNN-B)', column_MSE_1 , column_MSE_2, (0,1.5), (0,1.5), gridsize=60)
    plot_hist_2D(ax8, dataframe2, '(8) CNN-B: SSIM Bin Plot', column_SSIM_1, 'SSIM (CNN-B)', column_SSIM_1, column_SSIM_2, (.7,1), (.7,1), gridsize=100)

save_path = plot_dir+'figure-histograms.png'
savefig(save_path, bbox_inches='tight')


# Alter Dataset

## Sample/Trim/Resize

In [ ]:
# --- Refresh Repository ---
#refresh_repo(IN_COLAB, local_repo_path=local_repo_dirPath)

sample_trim_resize(
    input_dir='/content/drive/MyDrive/Colab/Working/dataset-sets',
    input_file='train-highCountSino-382x513.npy',
    output_dir='/content/drive/MyDrive/Colab/Working/dataset-sets',
    output_file='train-highCountSino-320x257.npy',
    sample_division=1,
    remove_n=0,
    crop_h=320,
    crop_w=None,
    new_height=None,
    new_width=None,
    pool_h_factor=None,
    pool_w_factor=2,
    seed=0,                # default seed for reproducibility
    log_file="log-SampleTrimResize.txt",
    skip_if_exists_vm=False,
    dry_run=False
)

## Precompute Attenuation Sinograms

In [ ]:
precompute=True

#atten_image_fileName = 'train-attenMap.npy'
#atten_sino_fileName = 'train-attenSino-382x513.npy'
atten_image_fileName = 'test-attenMap.npy'
atten_sino_fileName = 'test-attenSino-382x513.npy'


if precompute:
    precompute_atten_sinos(
        project_dirPath,
        data_dirName,
        atten_image_fileName,
        atten_sino_fileName,
        sino_height=382,
        sino_width=512,
        theta_type='symmetrical',
        atten_creation_pool_size=2,
        circle=False,
    )

## Sort: Dataset by Metric

In [ ]:
## Changeable Variables ##

load_sino_path = '/content/drive/MyDrive/Repository/PET_Data/train_sino-70k.npy'
load_image_path = '/content/drive/MyDrive/Repository/PET_Data/train_image-70k.npy'
save_sino_path = '/content/drive/MyDrive/Repository/PET_Data/quartile_data/train_sino-lowSSIM-17500.npy'
save_image_path = '/content/drive/MyDrive/Repository/PET_Data/quartile_data/train_image-lowSSIM-17500.npy'
'''
metric_function = MSE
max_save_index = 17500
threshold = 0.330922
threshold_min_max = 'min'
'''
metric_function = SSIM
max_save_index = 17500
threshold = 0.837850 #0.837850  # MSE (min): 0.330922, SSIM (max): 0.837850
threshold_min_max = 'max'

## Run & Verify Result ##
save_sino_array, save_image_array = sort_DataSet(config, load_image_path, load_sino_path, save_image_path, save_sino_path, max_save_index,
                                                 metric_function, threshold, threshold_min_max=threshold_min_max, num_examples=-1, visualize=False, sino_scale=sino_scale)

sino_ground_scaled

## Sort: Check & Save

In [ ]:
#save_sino_path = '/content/drive/MyDrive/Repository/PET_Data/quartile_data/train_sino-lowSSIM-17500.npy'
#save_image_path = '/content/drive/MyDrive/Repository/PET_Data/quartile_data/train_sino-lowSSIM-17500.npy'

# Print sorted array shape & display a few images #
print('save_sino_array.shape: ', save_sino_array.shape)
print('save_image_array.shape: ', save_image_array.shape)

print('save_sino_array sample images')
print('save_image_array sample images')
show_multiple_matched_tensors(torch.from_numpy(save_sino_array[500:509]))
show_multiple_matched_tensors(torch.from_numpy(save_image_array[500:509]))


# Save the sorted array to disk #
save_sino_array.flush()
save_image_array.flush()
#np.save(save_sino_path, save_sino_array)
#np.save(save_image_path, save_image_array)

# Load the saved array and make sure it's the same size/has the same images #
load_sino_array = np.load(save_sino_path, mmap_mode='r')
load_image_array = np.load(save_image_path, mmap_mode='r')
print('load_sino_array.shape: ', load_sino_array.shape)
print('load_image_array.shape: ', load_image_array.shape)

print('load_sino_array sample images')
print('load_image_array sample images')
show_multiple_matched_tensors(torch.from_numpy(load_sino_array[500:509]))
show_multiple_matched_tensors(torch.from_numpy(load_image_array[500:509]))


# Experimenting

In [ ]:
## Find what GPU I'm using ##
import torch
print(torch.cuda.is_available())  # Should be True

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# Notes:
Change next
===========
Change out HyperOpt search for Optuna
Install Git on home machine: https://git-scm.com/install/windows





For high/low MSE experiments
============================
-Tuned networks for 180 minutes each.

-Trained for 100 epochs using on-the-fly augmentation

-See notes in checkpoint folder


For LDM, window = 5, stride = 2
===============================
tune_max_t = 20            

tune_minutes = 180      

tune_batches_per_report=12    

tune_augment=False

Tensor board works for all experiments except the last one.
My plotting function no longer works for any of the experiments.